# API

In [1]:
import pandas as pd
import numpy as np
import json
from datetime import datetime
import requests

First look for the end point - so you can use the get 

In [2]:
endpoint = '/api/mobile/sessions.json'

In [3]:
root = 'http://aircasting.habitatmap.org'

In [5]:
seconds_since_epoch = datetime.now().timestamp()
int(seconds_since_epoch)
# You can look at the docstring for datetime, right click, open up console for this notebook
# then at in ? 

1657044323

In [6]:
#Year, month, day, hour, minute, second
time_to = int(datetime(2022, 7, 5, 16,43,0).timestamp())
time_from = int(datetime(2018, 7, 5, 16,43,0).timestamp())

In [7]:
#Latitude and longitude:
west = -86.788254
south = 29.206719
east = -76.421585
north = 49.415508

In [8]:
useragent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'

In [21]:
#By taking the example from the github page that documents the API, we took the parameters

parameters = {"time_from":time_from,
"time_to":time_to,
"tags":"",
"usernames":"",
"west": west,
"east": east,
"south": south,
"north":north,
"limit":100,
"offset":0,
"sensor_name":"airbeam2-pm2.5",
"measurement_type":"Particulate Matter",
"unit_symbol":"µg/m³"}

We pulled the root from this, as we can see where the endpoint begins before the ?q, which the parameters are there after

http://aircasting.habitatmap.org
/api/mobile/sessions.json?q=
{"time_from":1533081600,
"time_to":1535759940,
"tags":"",
"usernames":"HabitatMap",
"west":-73.96594349649638,
"east":-73.9480263402769,
"south":40.70849431072799,
"north":40.713585287529995,
"limit":1,
"offset":0,
"sensor_name":"airbeam2-pm2.5",
"measurement_type":"Particulate Matter",
"unit_symbol":"µg/m³"}

One of the parameters is time, which is shown as unix time or seconds since epoch (Jan 1, 1970)

These times are given at the prime meridian 

In [22]:
# then you need to use the json.dumps in order to get the "" that was needed
json.dumps(parameters)

'{"time_from": 1530834180, "time_to": 1657064580, "tags": "", "usernames": "", "west": -86.788254, "east": -76.421585, "south": 29.206719, "north": 49.415508, "limit": 100, "offset": 0, "sensor_name": "airbeam2-pm2.5", "measurement_type": "Particulate Matter", "unit_symbol": "\\u00b5g/m\\u00b3"}'

In [23]:
# need to build a new dictionary to have the q included, which is in the format
newdict = {'q' : json.dumps(parameters)}
newdict

{'q': '{"time_from": 1530834180, "time_to": 1657064580, "tags": "", "usernames": "", "west": -86.788254, "east": -76.421585, "south": 29.206719, "north": 49.415508, "limit": 100, "offset": 0, "sensor_name": "airbeam2-pm2.5", "measurement_type": "Particulate Matter", "unit_symbol": "\\u00b5g/m\\u00b3"}'}

In [24]:
# At first we left the parameters blank and then went in to fill them back

r = requests.get(root + endpoint,
                 headers = {'User-agent':useragent},
                 params = newdict)
r

<Response [200]>

In [25]:
json.loads(r.text)

{'sessions': [{'id': 1717086,
   'title': 'i',
   'start_time_local': '2021-01-30T23:21:22.000Z',
   'end_time_local': '2021-01-30T23:46:24.000Z',
   'type': 'MobileSession',
   'username': 'Devin J',
   'streams': {'AirBeam2-PM2.5': {'average_value': 21.3389,
     'id': 2097514,
     'max_latitude': 37.5224108,
     'max_longitude': -77.3964754,
     'measurement_short_type': 'PM',
     'measurement_type': 'Particulate Matter',
     'measurements_count': 1502,
     'min_latitude': 37.5221928,
     'min_longitude': -77.396713,
     'sensor_name': 'AirBeam2-PM2.5',
     'sensor_package_name': 'AirBeam2:001896000326',
     'session_id': 1717086,
     'size': 1502,
     'start_latitude': 37.5222549,
     'start_longitude': -77.3964754,
     'threshold_high': 55,
     'threshold_low': 12,
     'threshold_medium': 35,
     'threshold_very_high': 150,
     'threshold_very_low': 0,
     'unit_name': 'micrograms per cubic meter',
     'unit_symbol': 'µg/m³'}}},
  {'id': 1691939,
   'title': '6

In [27]:
sessiondf = pd.json_normalize(json.loads(r.text), record_path = 'sessions')

In [28]:
sessiondf.columns

Index(['id', 'title', 'start_time_local', 'end_time_local', 'type', 'username',
       'streams.AirBeam2-PM2.5.average_value', 'streams.AirBeam2-PM2.5.id',
       'streams.AirBeam2-PM2.5.max_latitude',
       'streams.AirBeam2-PM2.5.max_longitude',
       'streams.AirBeam2-PM2.5.measurement_short_type',
       'streams.AirBeam2-PM2.5.measurement_type',
       'streams.AirBeam2-PM2.5.measurements_count',
       'streams.AirBeam2-PM2.5.min_latitude',
       'streams.AirBeam2-PM2.5.min_longitude',
       'streams.AirBeam2-PM2.5.sensor_name',
       'streams.AirBeam2-PM2.5.sensor_package_name',
       'streams.AirBeam2-PM2.5.session_id', 'streams.AirBeam2-PM2.5.size',
       'streams.AirBeam2-PM2.5.start_latitude',
       'streams.AirBeam2-PM2.5.start_longitude',
       'streams.AirBeam2-PM2.5.threshold_high',
       'streams.AirBeam2-PM2.5.threshold_low',
       'streams.AirBeam2-PM2.5.threshold_medium',
       'streams.AirBeam2-PM2.5.threshold_very_high',
       'streams.AirBeam2-PM2.5.t

In [32]:
streamID = sessiondf['streams.AirBeam2-PM2.5.id']

In [33]:
endpoint2 = '/api/measurements.json'
params2 = {'stream_ids': streamID}